## 0. 기본 셋팅

In [1]:
import pandas as pd
import numpy as np
table_raw = pd.read_csv('./meerkat_raw.csv', skiprows=1)

In [2]:
# 기본정보 파악
table_raw.head(1)

,날짜,기록 시간,기록자,호선,탑승역,하차역,지나온 역 수,시간,붐빔 정도,노약자석 빈자리 여부,성별,비어있었던 역 수,연령대,뱃지 여부,임산부 여부,무거운 짐 여부,행동,기타 특이사항,짐 종류
0,2019. 7. 31.,19:10:00,재민,5,공덕,여의도,3,6.0,N,NaN,여,NaN,중년,X,X,O,"이어폰, 핸드폰",NaN,"쇼핑백, 장우산"


In [3]:
len(table_raw)

833

In [4]:
# 컬럼 확인
table_raw.columns

Index(['날짜', '기록 시간', '기록자', '호선', '탑승역', '하차역', '지나온 역 수', '시간', '붐빔 정도',
       '노약자석 빈자리 여부', '성별', '비어있었던 역 수', '연령대', '뱃지 여부', '임산부 여부', '무거운 짐 여부',
       '행동', '기타 특이사항', '짐 종류'],
      dtype='object')

In [5]:
# raw data는 그대로 남겨두고 복사본으로 정제할 계획
table = table_raw.copy()

## 1. 사용하지 않을 컬럼 삭제
: '기록자', '지나온 역 수', '시간', '기타 특이사항', '짐 종류'

In [6]:
table = table.drop(['기록자', '지나온 역 수', '시간', '기타 특이사항', '짐 종류'], axis=1)

In [7]:
table.head(1)

,날짜,기록 시간,호선,탑승역,하차역,붐빔 정도,노약자석 빈자리 여부,성별,비어있었던 역 수,연령대,뱃지 여부,임산부 여부,무거운 짐 여부,행동
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,X,X,O,"이어폰, 핸드폰"


## 2. 컬럼명 영어로 변경
: date, time, line, start, stop, crowdedness, is_old_seat_empty, sex, p_seat_empty_count, age, badge, pregnant, heavy, behavior

In [8]:
table = table.rename(columns = {
    '날짜': 'date',
    '기록 시간': 'time',
    '호선': 'line',
    '탑승역': 'start',
    '하차역': 'stop',
    '붐빔 정도': 'crowdedness',
    '노약자석 빈자리 여부': 'is_old_seat_empty',
    '성별': 'sex',
    '비어있었던 역 수': 'p_seat_empty_count',
    '연령대': 'age',
    '뱃지 여부': 'badge',
    '임산부 여부': 'pregnant',
    '무거운 짐 여부': 'heavy',
    '행동': 'behavior'
})

# is_p_seat_empty  추가 sex 오른쪽에

In [9]:
table.head(1)

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,X,X,O,"이어폰, 핸드폰"


## 3. O, X 데이터를 숫자 1, 0으로 변환
- 노약자석 빈자리 여부 [is_old_seat_empty]
    - O: 1 (빈자리가 있다), X: 0 (빈자리가 없다) 
- 뱃지 여부 [badge]
    - O: 1 (뱃지가 있다), X: 0 (뱃지가 없다) 
- 임산부 여부 [pregnant]
    - O: 1 (임산부다), X, ?: 0 (임산부가 아니다) 
- 무거운 짐 여부 [heavy] 컬럼 
    - O: 1 (무거운 짐을 들었다), X: 0 (무거운 짐을 안 들었다) 

In [10]:
table[['is_old_seat_empty', 'badge', 'pregnant', 'heavy']] = table[['is_old_seat_empty', 'badge', 'pregnant', 'heavy']].replace({'O': 1, 'X': 0, '?':0})

In [11]:
table['is_old_seat_empty'].unique()

array([nan, 0, 1, 'N'], dtype=object)

In [12]:
# 붐빔 정도(crowdedness)에 N을 적으면서 노약자석 빈자리 여부(is_old_seat_empty)에도 똑같이 N을 적은 행이 4개 있다.
table[table['is_old_seat_empty']=='N']

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior
678,2019. 8. 13.,19:08:00,1,신길,온수,N,N,여,NaN,청년,1.0,1.0,1.0,핸드폰
679,2019. 8. 13.,19:08:00,1,신길,온수,N,N,남,NaN,청년,0.0,0.0,0.0,핸드폰
680,2019. 8. 13.,19:16:00,1,구로,온수,N,N,비었음,4.0,NaN,NaN,NaN,NaN,NaN
681,2019. 8. 13.,19:16:00,1,신길,온수,N,N,여,NaN,중년,0.0,0.0,1.0,핸드폰


In [13]:
# 이 4개의 데이터를 일단 missing data(결측치)처럼 처리해놓고 나중에 다른 결측치와 동시에 채울 예정
table.loc[678:681, 'is_old_seat_empty'] = table.loc[678:681, 'is_old_seat_empty'].replace('N', np.nan)

In [14]:
# 현재 노약자석 빈자리 여부의 결측치 총 17개
table[table['is_old_seat_empty'].isnull()]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,0.0,0.0,1.0,"이어폰, 핸드폰"
1,2019. 7. 31.,19:21:00,9,여의도,신논현,F,NaN,여,NaN,중년,0.0,0.0,0.0,핸드폰
2,2019. 7. 31.,19:28:00,9,여의도,신논현,F,NaN,여,NaN,청년,0.0,0.0,0.0,핸드폰
3,2019. 7. 31.,19:32:00,9,여의도,신논현,F,NaN,여,NaN,청년,0.0,1.0,0.0,핸드폰
4,2019. 7. 31.,22:38:00,10,선정릉,야탑,F,NaN,여,NaN,청년,1.0,1.0,0.0,"핸드폰, 대화"
5,2019. 7. 31.,22:42:00,10,선정릉,야탑,F,NaN,남,NaN,중년,0.0,0.0,0.0,핸드폰
6,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,중년,0.0,0.0,0.0,NaN
7,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,청년,0.0,0.0,0.0,핸드폰
8,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,중년,0.0,0.0,0.0,핸드폰
9,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,중년,0.0,0.0,0.0,NaN


In [15]:
table['badge'].unique()

array([ 0.,  1., nan])

In [16]:
table['pregnant'].unique()

array([ 0.,  1., nan])

In [17]:
table['heavy'].unique()

array([ 1.,  0., nan])

In [18]:
# 무거운 짐 여부(heavy) 결측치 1개
table[~(table['sex']=='비었음') & (table['heavy'].isnull())]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior
505,2019. 8. 10.,22:49:00,7,대림,신중동,E,1,남,NaN,노년,0.0,0.0,NaN,대화


In [19]:
# 짐 종류 기록했던 것을 확인하기 위하여 raw data에서 index:505 인 행을 찾는다.
table_raw.iloc[505, :]

날짜             2019. 8. 10.
기록 시간              22:49:00
기록자                      단비
호선                        7
탑승역                      대림
하차역                     신중동
지나온 역 수                  10
시간                      NaN
붐빔 정도                     E
노약자석 빈자리 여부               O
성별                        남
비어있었던 역 수               NaN
연령대                      노년
뱃지 여부                     X
임산부 여부                    X
무거운 짐 여부                NaN
행동                       대화
기타 특이사항                 NaN
짐 종류                    NaN
Name: 505, dtype: object

In [20]:
# 짐 종류 기록이 없으므로 짐을 가지고 있지 않은 사람으로 간주하고 heavy에 0을 넣는다.
table.loc[505, ['heavy']] = table.loc[505, ['heavy']].fillna(0)

In [21]:
table.loc[505,:]

date                  2019. 8. 10.
time                      22:49:00
line                             7
start                           대림
stop                           신중동
crowdedness                      E
is_old_seat_empty                1
sex                              남
p_seat_empty_count             NaN
age                             노년
badge                            0
pregnant                         0
heavy                            0
behavior                        대화
Name: 505, dtype: object

In [22]:
table.head(1)

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,0.0,0.0,1.0,"이어폰, 핸드폰"


## 4. 붐빔 정도 컬럼 값 기준으로 3개 컬럼 추가 생성
- E, N, F: 각각 해당하는 컬럼에 1, 해당하지 않는 컬럼에 0

In [23]:
table['crowdedness'].unique()

array(['N', 'F', 'E'], dtype=object)

In [24]:
table['E'] = table['crowdedness'].replace({'E': 1, 'N': 0, 'F': 0})

In [25]:
table['N'] = table['crowdedness'].replace({'E': 0, 'N': 1, 'F': 0})

In [26]:
table['F'] = table['crowdedness'].replace({'E': 0, 'N': 0, 'F': 1})

In [27]:
table.head(1)

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior,E,N,F
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,0.0,0.0,1.0,"이어폰, 핸드폰",0,1,0


## 5. 성별 컬럼 값 기준으로 3개 컬럼 추가 생성 
- is_p_seat_empty: 임산부석이 비었을 때 1, 비어있지 않을 때 0
- female: 임산부석 착석자가 여성일 때 1, 남성일 때 0
- male: 임산부석 착석자가 남성일 때 1, 여성일 때 0

In [28]:
table['sex'].unique()

array(['여', '남', '비었음'], dtype=object)

In [29]:
table['is_p_seat_empty'] = table['sex'].replace({'여': 0, '남': 0, '비었음': 1})

In [30]:
table['female'] = table['sex'].replace({'여': 1, '남': 0, '비었음': 0})

In [31]:
table['male'] = table['sex'].replace({'여': 0, '남': 1, '비었음': 0})

In [32]:
table.head(1)

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,badge,pregnant,heavy,behavior,E,N,F,is_p_seat_empty,female,male
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,0.0,0.0,1.0,"이어폰, 핸드폰",0,1,0,0,1,0


## 6. 연령대 컬럼 값 기준으로 5개 컬럼 추가 생성
: child(어린이), teenage(청소년), youth(청년), middle_age(중년), old_age(노년)

In [33]:
table['age'].unique()

array(['중년', '청년', '노년', nan, '어린이', '청소년'], dtype=object)

In [34]:
table['child'] = table['age'].replace({'중년': 0, '청년': 0, '노년': 0, '어린이': 1, '청소년': 0})

In [35]:
table['teenage'] = table['age'].replace({'중년': 0, '청년': 0, '노년': 0, '어린이': 0, '청소년': 1})

In [36]:
table['youth'] = table['age'].replace({'중년': 0, '청년': 1, '노년': 0, '어린이': 0, '청소년': 0})

In [37]:
table['middle_age'] = table['age'].replace({'중년': 1, '청년': 0, '노년': 0, '어린이': 0, '청소년': 0})

In [38]:
table['old_age'] = table['age'].replace({'중년': 0, '청년': 0, '노년': 1, '어린이': 0, '청소년': 0})

In [39]:
table.head(1)

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,N,F,is_p_seat_empty,female,male,child,teenage,youth,middle_age,old_age
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,...,1,0,0,1,0,0.0,0.0,0.0,1.0,0.0


## 7. 행동 컬럼 정리

### 7-1. 한 사람이 동시에 한 행동들을 각 컬럼으로 나누기

In [40]:
table['behavior'].unique()

array(['이어폰, 핸드폰', '핸드폰', '핸드폰, 대화', nan, '이어폰, 잠', '멍', '핸드폰 게임', '이어폰',
       'X', '잠', '대화', '화장', '아이와 대화', '잠, 이어폰', '공부', '독서',
       '이어폰, 핸드폰(영상시청)', '핸드폰, 이어폰', '미어캣', '전단지 정독', '부채질', '묵주 돌리기',
       '봉에 팔 걸침, 미어캣', '통화', '핸드폰 -> 잠', '음료마심', '핸드폰, 이어폰, 잠',
       '핸드폰 -> 독서', '전화', '먹기', '핸드폰, 손풍기쐬기', '빵먹음', '핸드폰, 주변 살피기',
       '친구와 대화', '필기', '손풍기', '핸드폰, 잠', '책', '핸드폰, 먹기', '핸드폰 ',
       '핸드폰(전화통화)', '핸드폰, 멍'], dtype=object)

In [41]:
# 하나의 값에 두 가지 이상의 행동이 기록된 데이터는 모두 쉼표로 구분하도록 변경한다.
table['behavior'] = table['behavior'].replace({'핸드폰 -> 독서': '핸드폰, 독서',
                                                                      '핸드폰 -> 잠': '핸드폰, 잠'}).str.strip()

In [42]:
# 임산부석에 앉은 사람이 있는데 행동 데이터가 기록되지 않은 결측치 20개 -> 차후 해결
len(table[(table['is_p_seat_empty']==0) & (table['behavior'].isnull())])

20

In [43]:
# Null 값 제외 (결측치 20개 + 임산부석 착석자가 없어서 행동 데이터가 없는 경우)
actual_behavior = table[~table['behavior'].isnull()]['behavior']

In [44]:
# 한 사람이 동시에 한 행동(최대 3개)을 쉼표로 구분해 behavior1, behavior2, behavior3 컬럼에 각각 넣는다.
table[['behavior1','behavior2','behavior3']] = actual_behavior.str.split(",",expand=True)

In [45]:
table.head()

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,female,male,child,teenage,youth,middle_age,old_age,behavior1,behavior2,behavior3
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,...,1,0,0.0,0.0,0.0,1.0,0.0,이어폰,핸드폰,None
1,2019. 7. 31.,19:21:00,9,여의도,신논현,F,NaN,여,NaN,중년,...,1,0,0.0,0.0,0.0,1.0,0.0,핸드폰,None,None
2,2019. 7. 31.,19:28:00,9,여의도,신논현,F,NaN,여,NaN,청년,...,1,0,0.0,0.0,1.0,0.0,0.0,핸드폰,None,None
3,2019. 7. 31.,19:32:00,9,여의도,신논현,F,NaN,여,NaN,청년,...,1,0,0.0,0.0,1.0,0.0,0.0,핸드폰,None,None
4,2019. 7. 31.,22:38:00,10,선정릉,야탑,F,NaN,여,NaN,청년,...,1,0,0.0,0.0,1.0,0.0,0.0,핸드폰,대화,None


### 7-2. 텍스트 정제 : 10가지로 카테고리화

In [46]:
table[['behavior1','behavior2','behavior3']] = table[['behavior1','behavior2','behavior3']].apply(lambda x: x.str.strip().replace({'핸드폰 게임': '핸드폰',
                                                                                                                                                                                                       '핸드폰(영상시청)': '핸드폰',
                                                                                                                                                                                                       '아이와 대화': '대화',
                                                                                                                                                                                                       '친구와 대화': '대화',
                                                                                                                                                                                                       'X': '멍',
                                                                                                                                                                                                       '미어캣': '멍',
                                                                                                                                                                                                       '주변 살피기': '멍',
                                                                                                                                                                                                       '묵주 돌리기': '멍',
                                                                                                                                                                                                       '봉에 팔 걸침': '멍',
                                                                                                                                                                                                       '부채질': '멍',
                                                                                                                                                                                                       '손풍기쐬기': '멍',
                                                                                                                                                                                                       '손풍기': '멍',
                                                                                                                                                                                                       '공부': '필기',
                                                                                                                                                                                                       '전단지 정독': '독서',
                                                                                                                                                                                                       '책': '독서',
                                                                                                                                                                                                       '빵먹음': '먹기',
                                                                                                                                                                                                       '음료마심': '먹기',
                                                                                                                                                                                                       '전화': '통화',
                                                                                                                                                                                                       '핸드폰(전화통화)': '통화'}))

In [47]:
table['behavior1'].unique()

array(['이어폰', '핸드폰', nan, '멍', '잠', '대화', '화장', '필기', '독서', '통화', '먹기'],
      dtype=object)

In [48]:
table['behavior2'].unique()

array(['핸드폰', None, '대화', nan, '잠', '이어폰', '멍', '독서', '먹기'], dtype=object)

In [49]:
table['behavior3'].unique()

array([None, nan, '잠'], dtype=object)

### 7-3. 카테고리 별로 나눠서 10개 컬럼 추가 생성

: 핸드폰(cellphone), 멍(thinking), 잠(sleep), 이어폰(earphone), 대화(talking), 통화(calling), 독서(reading), 화장(makeup), 먹기(eating), 필기(writing)

In [50]:
behavior_columns = ['핸드폰', '멍', '잠', '이어폰', '대화', '통화', '독서', '화장', '먹기', '필기']

In [51]:
for word in behavior_columns:
    table[word] = table['behavior1'].apply(lambda x: 1 if x == word else 0)
    table[word] += table['behavior2'].apply(lambda x: 1 if x == word else 0)
    table[word] += table['behavior3'].apply(lambda x: 1 if x == word else 0)

In [52]:
# 컬럼명 영어로 변경
table = table.rename(columns = {'핸드폰':'cellphone', 
                                                    '멍':'thinking', 
                                                    '잠':'sleep', 
                                                    '이어폰':'earphone', 
                                                    '대화':'talking', 
                                                    '통화':'calling', 
                                                    '독서':'reading', 
                                                    '화장':'makeup', 
                                                    '먹기':'eating', 
                                                    '필기':'writing'})

In [53]:
table.head()

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,...,1,0,0,1,0,0,0,0,0,0
1,2019. 7. 31.,19:21:00,9,여의도,신논현,F,NaN,여,NaN,중년,...,1,0,0,0,0,0,0,0,0,0
2,2019. 7. 31.,19:28:00,9,여의도,신논현,F,NaN,여,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
3,2019. 7. 31.,19:32:00,9,여의도,신논현,F,NaN,여,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
4,2019. 7. 31.,22:38:00,10,선정릉,야탑,F,NaN,여,NaN,청년,...,1,0,0,0,1,0,0,0,0,0


## 8. 결측치(missing data) 처리

### 8-1. 노약자석 빈자리 여부(is_old_seat_empty) 결측치

#### 1) 결측치를 제외한 테이블에서 붐빔 정도(crowdedness)에 따라 노약자석 빈자리 여부(is_old_seat_empty) 값의 비율을 각각 확인한다.
#### 2) 비율에 맞게 결측치를 채운다.

In [54]:
# 결측치 확인: 총 17개 중 10개는 crowdedness=N, 7개는 crowdedness=F
table[table['is_old_seat_empty'].isnull()]['crowdedness'].value_counts()

N    10
F     7
Name: crowdedness, dtype: int64

In [55]:
# 결측치를 제외한 데이터 중 crowdedness의 값이 N인 테이블을 만든다.
old_seat_n = table[(~table['is_old_seat_empty'].isnull()) & (table['crowdedness']=='N')]

In [56]:
# crowdedness=N 일 때 is_old_seat_empty 값의 비율 
print('is_old_seat_empty의 값이 0:', round(len(old_seat_n[old_seat_n['is_old_seat_empty']==0])/len(old_seat_n)*100, 2), '%')
print('is_old_seat_empty의 값이 1:', round(len(old_seat_n[old_seat_n['is_old_seat_empty']==1])/len(old_seat_n)*100, 2), '%')

is_old_seat_empty의 값이 0: 39.88 %
is_old_seat_empty의 값이 1: 60.12 %


In [57]:
table[table['is_old_seat_empty'].isnull() & (table['crowdedness']=='N')]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,NaN,여,NaN,중년,...,1,0,0,1,0,0,0,0,0,0
6,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
7,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
8,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,중년,...,1,0,0,0,0,0,0,0,0,0
9,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
10,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,NaN,남,NaN,노년,...,0,0,0,0,0,0,0,0,0,0
678,2019. 8. 13.,19:08:00,1,신길,온수,N,NaN,여,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
679,2019. 8. 13.,19:08:00,1,신길,온수,N,NaN,남,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
680,2019. 8. 13.,19:16:00,1,구로,온수,N,NaN,비었음,4.0,NaN,...,0,0,0,0,0,0,0,0,0,0
681,2019. 8. 13.,19:16:00,1,신길,온수,N,NaN,여,NaN,중년,...,1,0,0,0,0,0,0,0,0,0


In [58]:
# crowdedness=N인 결측치 10개 중 4개에 0을, 6개에 1을 넣는다.
# 0을 넣을 데이터 인덱스: 0, 6, 7, 8 
# 1을 넣을 데이터 인덱스: 9, 10, 678, 679, 680, 681

table.loc[[0,6,7,8], ['is_old_seat_empty']] = table.loc[[0,6,7,8], ['is_old_seat_empty']].fillna(0)
table.loc[[9, 10, 678, 679, 680, 681], ['is_old_seat_empty']] = table.loc[[9, 10, 678, 679, 680, 681], ['is_old_seat_empty']].fillna(1)

In [59]:
# 결측치를 제외한 데이터 중 crowdedness의 값이 F인 테이블을 만든다.
old_seat_f = table[(~table['is_old_seat_empty'].isnull()) & (table['crowdedness']=='F')]

In [60]:
# crowdedness=F 일 때 is_old_seat_empty 값의 비율 
print('is_old_seat_empty의 값이 0:', round(len(old_seat_f[old_seat_f['is_old_seat_empty']==0])/len(old_seat_f)*100, 2), '%')
print('is_old_seat_empty의 값이 1:', round(len(old_seat_f[old_seat_f['is_old_seat_empty']==1])/len(old_seat_f)*100, 2), '%')

is_old_seat_empty의 값이 0: 82.89 %
is_old_seat_empty의 값이 1: 17.11 %


In [61]:
# crowdedness=F인 결측치 7개 중 6개에 0을, 1개에 1을 넣는다.
print('0을 넣을 개수:', 82.89 / 100 * 7)
print('1을 넣을 개수:', 17.11 / 100 * 7)

0을 넣을 개수: 5.8023
1을 넣을 개수: 1.1977


In [62]:
table[table['is_old_seat_empty'].isnull()]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
1,2019. 7. 31.,19:21:00,9,여의도,신논현,F,NaN,여,NaN,중년,...,1,0,0,0,0,0,0,0,0,0
2,2019. 7. 31.,19:28:00,9,여의도,신논현,F,NaN,여,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
3,2019. 7. 31.,19:32:00,9,여의도,신논현,F,NaN,여,NaN,청년,...,1,0,0,0,0,0,0,0,0,0
4,2019. 7. 31.,22:38:00,10,선정릉,야탑,F,NaN,여,NaN,청년,...,1,0,0,0,1,0,0,0,0,0
5,2019. 7. 31.,22:42:00,10,선정릉,야탑,F,NaN,남,NaN,중년,...,1,0,0,0,0,0,0,0,0,0
11,2019. 7. 31.,18:47:00,5,공덕,신길,F,NaN,여,NaN,청년,...,1,0,0,1,0,0,0,0,0,0
12,2019. 7. 31.,18:47:00,5,공덕,신길,F,NaN,여,NaN,중년,...,0,0,1,1,0,0,0,0,0,0


In [63]:
# 0을 넣을 데이터 인덱스: 1, 2, 3, 4, 5, 11
# 1을 넣을 데이터 인덱스: 12

table.loc[[1, 2, 3, 4, 5, 11], ['is_old_seat_empty']] = table.loc[[1, 2, 3, 4, 5, 11], ['is_old_seat_empty']].fillna(0)
table.loc[12, ['is_old_seat_empty']] = table.loc[12, ['is_old_seat_empty']].fillna(1)

In [64]:
# 결측치 모두 채워진 것 확인
table[table['is_old_seat_empty'].isnull()]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing


### 8-2. 행동(behavior) 결측치

#### 1) 결측치를 제외한 테이블을 만든다.
#### 2) 동시에 여러 행동을 한 사람의 경우, 행동 데이터가 behavior1, behavior2, behavior3으로 나누어져 있다.
#### 3) 2개의 행동을 해서 behavior1과 behavior2에 값이 들어있는 행은 동일한 프로필을 가지고 행동만 다른 2명으로 간주한다. 
    ex) 중년, 여성, 핸드폰 & 이어폰 (1명) -> 중년, 여성, 핸드폰 / 중년, 여성, 이어폰 (총 2명)
#### 4) 이 때 2명 중 behavior2의 값을 가진 사람의 데이터를 행으로 추가한다.
#### 5) 3개의 행동을 해서 behavior3에 값이 들어있는 행의 경우도 마찬가지로 한다.
#### 6) behavior_all이라는 컬럼을 만들어 behavior1, behavior2, behavior3의 값을 모두 여기에 넣는다.
#### 7) 새로 추가한 행까지 포함한 테이블의 길이를 전체로 하고 연령대(age)에 따라 behavior_all 값의 비율을 구한다.
#### 8) 비율에 맞게 결측치를 채우고 (behavior1 컬럼에 채운다), cellphone, thinking 등의 파생된 컬럼에도 0 또는 1의 값을 넣는다.

In [65]:
# 결측치 확인 : 총 20개 중 10개는 age=중년, 5개는 age=노년, 5개는 age=청년
table[(table['is_p_seat_empty']==0) & (table['behavior'].isnull())]['age'].value_counts()

중년    10
청년     5
노년     5
Name: age, dtype: int64

In [66]:
# 결측치 제외 테이블 생성
behavior_table = table[(table['is_p_seat_empty']==0) & (~table['behavior'].isnull())]

In [67]:
behavior_table.columns

Index(['date', 'time', 'line', 'start', 'stop', 'crowdedness',
       'is_old_seat_empty', 'sex', 'p_seat_empty_count', 'age', 'badge',
       'pregnant', 'heavy', 'behavior', 'E', 'N', 'F', 'is_p_seat_empty',
       'female', 'male', 'child', 'teenage', 'youth', 'middle_age', 'old_age',
       'behavior1', 'behavior2', 'behavior3', 'cellphone', 'thinking', 'sleep',
       'earphone', 'talking', 'calling', 'reading', 'makeup', 'eating',
       'writing'],
      dtype='object')

In [68]:
# 비율 계산에 필요없는 행 제거
behavior_table = behavior_table.drop(['behavior', 'cellphone', 'thinking', 'sleep', 'earphone', 'talking', 'calling', 'reading', 'makeup', 'eating', 'writing'], axis=1)

In [69]:
# 1가지 이상의 행동을 하고 있는 사람의 수: 556명
# 2가지 이상의 행동을 하고 있는 사람의 수: 64명
# 3가지 이상의 행동을 하고 있는 사람의 수: 1명

print(len(behavior_table))
print(len(behavior_table[~behavior_table['behavior2'].isnull()]))
print(len(behavior_table[~behavior_table['behavior3'].isnull()]))

556
64
1


In [70]:
behavior_table.head()

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,female,male,child,teenage,youth,middle_age,old_age,behavior1,behavior2,behavior3
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,0,여,NaN,중년,...,1,0,0.0,0.0,0.0,1.0,0.0,이어폰,핸드폰,None
1,2019. 7. 31.,19:21:00,9,여의도,신논현,F,0,여,NaN,중년,...,1,0,0.0,0.0,0.0,1.0,0.0,핸드폰,None,None
2,2019. 7. 31.,19:28:00,9,여의도,신논현,F,0,여,NaN,청년,...,1,0,0.0,0.0,1.0,0.0,0.0,핸드폰,None,None
3,2019. 7. 31.,19:32:00,9,여의도,신논현,F,0,여,NaN,청년,...,1,0,0.0,0.0,1.0,0.0,0.0,핸드폰,None,None
4,2019. 7. 31.,22:38:00,10,선정릉,야탑,F,0,여,NaN,청년,...,1,0,0.0,0.0,1.0,0.0,0.0,핸드폰,대화,None


In [71]:
# 2가지 또는 3가지의 행동을 동시에 하고 있는 사람을 각각 2명 또는 3명으로 간주하고 65개의 행을 추가한다.
# 모든 행동 데이터를 behavior_all 컬럼으로 합친다.

behavior1_table = behavior_table.drop(['behavior2', 'behavior3'], axis=1).rename(columns = {'behavior1':'behavior_all'})
behavior2_table = behavior_table[~behavior_table['behavior2'].isnull()].drop(['behavior1', 'behavior3'], axis=1).rename(columns = {'behavior2':'behavior_all'})
behavior3_table = behavior_table[~behavior_table['behavior3'].isnull()].drop(['behavior1', 'behavior2'], axis=1).rename(columns = {'behavior3':'behavior_all'})
behavior_table_all = behavior1_table.append(behavior2_table, sort=False).append(behavior3_table, sort=False)

In [72]:
len(behavior_table_all)

621

In [73]:
behavior_table_all['behavior_all'].unique()

array(['이어폰', '핸드폰', '멍', '잠', '대화', '화장', '필기', '독서', '통화', '먹기'],
      dtype=object)

In [74]:
# 'age=중년'일 때 behavior_all 값의 비율 (비율은 원래 전체 인원 수인 556으로 나눠서 구한다.)
round(behavior_table_all[behavior_table_all['age']=='중년'].groupby('behavior_all').size().apply(lambda x: x / len(behavior_table[behavior_table['age']=='중년'])*100), 2).sort_values(ascending=False)

behavior_all
핸드폰    55.66
멍      18.55
잠      16.74
이어폰     9.05
대화      6.33
통화      1.36
독서      1.36
먹기      0.90
필기      0.45
dtype: float64

In [75]:
table[(table['is_p_seat_empty']==0) & (table['behavior'].isnull()) & (table['age']=='중년')]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
6,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,0,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
9,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
131,2019. 8. 3.,10:32:00,5,왕십리,광화문,N,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
132,2019. 8. 3.,10:32:00,5,왕십리,광화문,N,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
133,2019. 8. 3.,18:06:00,5,광화문,왕십리,F,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
135,2019. 8. 3.,18:11:00,5,광화문,왕십리,F,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
182,2019. 8. 5.,9:20:00,5,신길,공덕,N,0,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
408,2019. 8. 9.,8:50:00,6,화랑대,공덕,F,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
410,2019. 8. 9.,8:52:00,6,화랑대,공덕,F,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0
761,2019. 8. 15.,12:38:00,2,선릉,역삼,N,1,여,NaN,중년,...,0,0,0,0,0,0,0,0,0,0


In [76]:
# 'age=중년'인 결측치 10개 중 6개에 '핸드폰'을, 2개에 '멍'을, 2개에 '잠'을 넣는다.
# '핸드폰'을 넣을 데이터 인덱스: 6, 9, 131, 132, 133, 135
# '멍'을 넣을 데이터 인덱스: 182, 408
# '잠'을 넣을 데이터 인덱스: 410, 761

table.loc[[6, 9, 131, 132, 133, 135], ['behavior1']] = table.loc[[6, 9, 131, 132, 133, 135], ['behavior1']].fillna('핸드폰')
table.loc[[182, 408], ['behavior1']] = table.loc[[182, 408], ['behavior1']].fillna('멍')
table.loc[[410, 761], ['behavior1']] = table.loc[[410, 761], ['behavior1']].fillna('잠')

In [77]:
# 'age=노년'일 때 behavior_all 값의 비율 
round(behavior_table_all[behavior_table_all['age']=='노년'].groupby('behavior_all').size().apply(lambda x: x / len(behavior_table[behavior_table['age']=='노년'])*100), 2).sort_values(ascending=False)

behavior_all
멍      46.32
핸드폰    25.26
잠      20.00
대화      5.26
이어폰     4.21
독서      3.16
필기      1.05
통화      1.05
dtype: float64

In [78]:
# 'age=중년'인 결측치 5개 중 3개에 '멍'을, 1개에 '핸드폰'을, 1개에 '잠'을 넣는다.
print(46.32 / 100 * 5)
print(25.26 / 100 * 5)
print(20 / 100 * 5)

2.316
1.263
1.0


In [79]:
table[(table['is_p_seat_empty']==0) & (table['behavior'].isnull()) & (table['age']=='노년')]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
10,2019. 7. 31.,18:30:00,6,공덕,화랑대,N,1,남,NaN,노년,...,0,0,0,0,0,0,0,0,0,0
104,2019. 8. 3.,10:13:00,1,병점,종각,E,1,여,NaN,노년,...,0,0,0,0,0,0,0,0,0,0
136,2019. 8. 3.,18:15:00,5,광화문,왕십리,F,1,여,NaN,노년,...,0,0,0,0,0,0,0,0,0,0
181,2019. 8. 5.,9:20:00,5,신길,공덕,F,0,남,NaN,노년,...,0,0,0,0,0,0,0,0,0,0
206,2019. 8. 5.,8:55:00,6,화랑대,공덕,N,0,여,NaN,노년,...,0,0,0,0,0,0,0,0,0,0


In [80]:
# '멍'을 넣을 데이터 인덱스: 10, 104, 136
# '핸드폰'을 넣을 데이터 인덱스: 181
# '잠'을 넣을 데이터 인덱스: 206

table.loc[[10, 104, 136], ['behavior1']] = table.loc[[10, 104, 136], ['behavior1']].fillna('멍')
table.loc[181, ['behavior1']] = table.loc[181, ['behavior1']].fillna('핸드폰')
table.loc[206, ['behavior1']] = table.loc[206, ['behavior1']].fillna('잠')

In [81]:
# 'age=청년'일 때 behavior_all 값의 비율 
round(behavior_table_all[behavior_table_all['age']=='청년'].groupby('behavior_all').size().apply(lambda x: x / len(behavior_table[behavior_table['age']=='청년'])*100), 2).sort_values(ascending=False)

behavior_all
핸드폰    71.75
이어폰    15.70
잠      14.80
멍       5.38
대화      3.14
통화      1.79
화장      1.35
먹기      0.90
필기      0.45
독서      0.45
dtype: float64

In [82]:
# 'age=청년'인 결측치 5개 중 3개에 '핸드폰'을, 1개에 '이어폰'을, 1개에 '잠'을 넣는다.
print(71.75 / 100 * 5)
print(15.70 / 100 * 5)
print(14.80 / 100 * 5)

3.5875000000000004
0.785
0.7400000000000001


In [83]:
table[(table['is_p_seat_empty']==0) & (table['behavior'].isnull()) & (table['age']=='청년')]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
134,2019. 8. 3.,18:06:00,5,광화문,왕십리,F,1,여,NaN,청년,...,0,0,0,0,0,0,0,0,0,0
180,2019. 8. 5.,9:15:00,1,병점,신길,F,1,남,NaN,청년,...,0,0,0,0,0,0,0,0,0,0
205,2019. 8. 5.,8:55:00,6,화랑대,공덕,N,0,여,NaN,청년,...,0,0,0,0,0,0,0,0,0,0
366,2019. 8. 8.,17:52:00,6,공덕,화랑대,N,1,여,NaN,청년,...,0,0,0,0,0,0,0,0,0,0
714,2019. 8. 14.,18:28:00,5,공덕,신길,F,0,여,NaN,청년,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# '핸드폰'을 넣을 데이터 인덱스: 134, 180, 205
# '이어폰'을 넣을 데이터 인덱스: 366
# '잠'을 넣을 데이터 인덱스: 714

table.loc[[134, 180, 205], ['behavior1']] = table.loc[[134, 180, 205], ['behavior1']].fillna('핸드폰')
table.loc[366, ['behavior1']] = table.loc[206, ['behavior1']].fillna('이어폰')
table.loc[714, ['behavior1']] = table.loc[206, ['behavior1']].fillna('잠')

In [85]:
# behavior1의 결측치 모두 채워진 것 확인
table[(table['is_p_seat_empty']==0) & (table['behavior1'].isnull())]

,date,time,line,start,stop,crowdedness,is_old_seat_empty,sex,p_seat_empty_count,age,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing


In [86]:
# '핸드폰'으로 채운 데이터 인덱스: 6, 9, 131, 132, 133, 135, 181, 134, 180, 205 -> cellphone 컬럼 값 1로 변경
# '멍'으로 채운 데이터 인덱스: 182, 408, 10, 104, 136 -> thinking 컬럼 값 1로 변경
# '잠'으로 채운 데이터 인덱스: 410, 761, 206, 714 -> sleep 컬럼 값 1로 변경
# '이어폰'으로 채운 데이터 인덱스: 366 -> earphone 컬럼 값 1로 변경

table.loc[[6, 9, 131, 132, 133, 135, 181, 134, 180, 205], ['cellphone']] = table.loc[[6, 9, 131, 132, 133, 135, 181, 134, 180, 205], ['cellphone']].replace(0, 1)
table.loc[[182, 408, 10, 104, 136], ['thinking']] = table.loc[[182, 408, 10, 104, 136], ['thinking']].replace(0, 1)
table.loc[[410, 761, 206, 714], ['sleep']] = table.loc[[410, 761, 206, 714], ['sleep']].replace(0, 1)
table.loc[366, ['earphone']] = table.loc[366, ['earphone']].replace(0, 1)

## 9. 컬럼 순서 정렬 후 csv 파일로 저장

In [87]:
table.columns

Index(['date', 'time', 'line', 'start', 'stop', 'crowdedness',
       'is_old_seat_empty', 'sex', 'p_seat_empty_count', 'age', 'badge',
       'pregnant', 'heavy', 'behavior', 'E', 'N', 'F', 'is_p_seat_empty',
       'female', 'male', 'child', 'teenage', 'youth', 'middle_age', 'old_age',
       'behavior1', 'behavior2', 'behavior3', 'cellphone', 'thinking', 'sleep',
       'earphone', 'talking', 'calling', 'reading', 'makeup', 'eating',
       'writing'],
      dtype='object')

In [88]:
table = table.drop(['sex', 'behavior'], axis=1)

In [89]:
table = table[['date', 'time', 'line', 'start', 'stop', 
                    'crowdedness', 'E', 'N', 'F', 'is_old_seat_empty', 
                    'is_p_seat_empty', 'p_seat_empty_count', 'female', 'male',
                    'age', 'child', 'teenage', 'youth', 'middle_age', 'old_age',
                    'badge', 'pregnant', 'heavy', 
                    'behavior1', 'behavior2', 'behavior3', 'cellphone', 'thinking', 'sleep', 'earphone', 'talking', 'calling', 'reading', 'makeup', 'eating', 'writing']]

In [90]:
table.head(1)

,date,time,line,start,stop,crowdedness,E,N,F,is_old_seat_empty,...,cellphone,thinking,sleep,earphone,talking,calling,reading,makeup,eating,writing
0,2019. 7. 31.,19:10:00,5,공덕,여의도,N,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0


In [91]:
table.to_csv('./meerkat_processed.csv')